In [1]:
# Including required python libraries used in this project
import numpy as np
import pandas as pd
import emoji

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, SimpleRNN,LSTM, Activation
from keras.utils import np_utils

import matplotlib.pyplot as plt

In [2]:
# reading train emoji and test emoji data which are attached in the repo
train = pd.read_csv('train_emoji.csv',header=None)
test = pd.read_csv('test_emoji.csv',header=None)

In [3]:
# Checking data by showing first 5 rows of the train data
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [4]:
# Checking data by showing first 5 rows of the test data
test.head()

,0,1
0,I want to eat\t,4
1,he did not answer\t,3
2,he got a raise\t,2
3,she got me a present\t,0
4,ha ha ha it was so funny\t,2


In [5]:
# Creating dictionary for some emoji's, consisting of key - number and value - emoji 
emoji_dict = { 0 : ":heart:", 1 : ":baseball:", 2 : ":smile:", 3 : ":disappointed:", 4 : ":fork_and_knife:"}

In [6]:
# Printing each emoji icon by emojizing each emoji
for ix in emoji_dict.keys():
    print (ix,end=" ")
    print (emoji.emojize(emoji_dict[ix], use_aliases=True))

0 ❤
1 ⚾
2 😄
3 😞
4 🍴


In [7]:
# Creating training and testing data
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

print (X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
print ("-------------------------")
print (X_train[0],Y_train[0])

(132,) (132,) (56,) (56,)
-------------------------
never talk to me again 3


In [8]:
# Splitting the train data from sentences to words
for ix in range(X_train.shape[0]):
    X_train[ix] = X_train[ix].split()

# Splitting the test data from sentences to words
for ix in range(X_test.shape[0]):
    X_test[ix] = X_test[ix].split()
    
# Converting labels into categorical form
Y_train = np_utils.to_categorical(Y_train)

<ipython-input-8-1de645c59088>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[ix] = X_train[ix].split()
<ipython-input-8-1de645c59088>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[ix] = X_test[ix].split()


In [9]:
# Now checking the above conversion by printing train and test data at 0th index
print (X_train[0],Y_train[0])

['never', 'talk', 'to', 'me', 'again'] [0. 0. 0. 1. 0.]


In [10]:
# To check maximum length of sentence in training data
np.unique(np.array([len(ix) for ix in X_train]) , return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
 array([ 4,  5, 26, 35, 20, 21, 11,  5,  1,  4], dtype=int64))

In [11]:
# To check maximum length of senetence in testing data
np.unique(np.array([len(ix) for ix in X_test]) , return_counts=True)

(array([2, 3, 4, 5, 6, 7, 8]),
 array([ 3, 12, 16, 17,  3,  4,  1], dtype=int64))

In [12]:
# Creating  embeddings dictionary with key = word and value = list of words in glove vector
embeddings_index = {}

f = open(r'D:\Marco\Universita\3ANNO\Tesi\emoji-prediction\glove.6B.50d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [13]:
# Checking length of a particular word
embeddings_index["i"].shape

(50,)

In [14]:
from scipy import spatial
# Checking cosine similarity of words happy and sad
spatial.distance.cosine(embeddings_index["happy"], embeddings_index["sad"])

0.31093674898147583

In [15]:
# Checking cosine similarity of words India and Delhi
spatial.distance.cosine(embeddings_index["india"], embeddings_index["delhi"])

0.18572336435317993

In [16]:
# Checking cosine similarity of words france and paris
spatial.distance.cosine(embeddings_index["france"], embeddings_index["paris"])

0.19746702909469604

In [17]:
# Filling the embedding matrix
embedding_matrix_train = np.zeros((X_train.shape[0], 10, 50))
embedding_matrix_test = np.zeros((X_test.shape[0], 10, 50))

for ix in range(X_train.shape[0]):
    for ij in range(len(X_train[ix])):
        embedding_matrix_train[ix][ij] = embeddings_index[X_train[ix][ij].lower()]
        
for ix in range(X_test.shape[0]):
    for ij in range(len(X_test[ix])):
        embedding_matrix_test[ix][ij] = embeddings_index[X_test[ix][ij].lower()]        

In [18]:
print (embedding_matrix_train.shape, embedding_matrix_test.shape)

(132, 10, 50) (56, 10, 50)


# Training model

## - Using RNN

In [19]:
# A simple RNN network to classify the emoji class from an input Sentence

model = Sequential()
model.add(SimpleRNN(64, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(SimpleRNN(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 10, 64)            7360      
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
_________________________________________________________________
activation (Activation)      (None, 5)                 0         
Total params: 15,941
Trainable params: 15,941
Non-trainable params: 0
____________________________________________________

In [20]:
# Setting Loss and Optimiser for the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# Training of the model and Setting hyperparameters for the model
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
5/5 [==============================] - 0s 5ms/step - loss: 1.9275 - accuracy: 0.2121
Epoch 2/50
5/5 [==============================] - 0s 5ms/step - loss: 1.7695 - accuracy: 0.2500
Epoch 3/50
5/5 [==============================] - 0s 4ms/step - loss: 1.6974 - accuracy: 0.2652
Epoch 4/50
5/5 [==============================] - 0s 4ms/step - loss: 1.5752 - accuracy: 0.3409
Epoch 5/50
5/5 [==============================] - 0s 4ms/step - loss: 1.5453 - accuracy: 0.3485
Epoch 6/50
5/5 [==============================] - 0s 4ms/step - loss: 1.5882 - accuracy: 0.3485
Epoch 7/50
5/5 [==============================] - 0s 3ms/step - loss: 1.4298 - accuracy: 0.3864
Epoch 8/50
5/5 [==============================] - 0s 4ms/step - loss: 1.4433 - accuracy: 0.4015
Epoch 9/50
5/5 [==============================] - 0s 4ms/step - loss: 1.3393 - accuracy: 0.4621
Epoch 10/50
5/5 [==============================] - 0s 4ms/step - loss: 1.2537 - accuracy: 0.5000
Epoch 11/50
5/5 [======================

In [22]:
# Prediction of the trained model 
pred = model.predict_classes(embedding_matrix_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


### ACCURACY

In [23]:
# Calculating the accuracy of the algorithm
float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.4107142857142857

In [24]:
# Printing the sentences with the predicted and labled emoji
for ix in range(embedding_matrix_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        print(ix)
        print (test[0][ix],end=" ")
        print (emoji.emojize(emoji_dict[pred[ix]], use_aliases=True),end=" ")
        print (emoji.emojize(emoji_dict[Y_test[ix]], use_aliases=True))

2
['he', 'got', 'a', 'raise'] ❤ 😄
3
['she', 'got', 'me', 'a', 'present'] 🍴 ❤
5
['he', 'is', 'a', 'good', 'friend'] 😄 ❤
6
['I', 'am', 'upset'] 😄 ❤
7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 🍴 ❤
9
['Stop', 'making', 'this', 'joke', 'ha', 'ha', 'ha'] 😞 😄
11
['work', 'is', 'hard'] ❤ 😞
12
['This', 'girl', 'is', 'messing', 'with', 'me'] 😄 😞
13
['are', 'you', 'serious', 'ha', 'ha'] 😞 😄
17
['Congratulation', 'for', 'having', 'a', 'baby'] 🍴 😄
18
['stop', 'messing', 'around'] ⚾ 😞
21
['you', 'brighten', 'my', 'day'] 🍴 😄
23
['she', 'is', 'a', 'bully'] 😄 😞
25
['I', 'am', 'upset'] 😄 😞
26
['I', 'worked', 'during', 'my', 'birthday'] 🍴 😞
27
['My', 'grandmother', 'is', 'the', 'love', 'of', 'my', 'life'] 🍴 ❤
28
['enjoy', 'your', 'break'] ⚾ 😄
29
['valentine', 'day', 'is', 'near'] 😄 ❤
30
['I', 'miss', 'you', 'so', 'much'] 😞 ❤
31
['throw', 'the', 'ball'] 😞 ⚾
32
['My', 'life', 'is', 'so', 'boring'] 😄 😞
34
['will', 'you', 'be', 'my', 'valentine'] 🍴 ❤
35
['he', 'can', 'pitch', 'really', 'well'

In [25]:
# Predicting for our random sentence
x = ['i', 'do', 'think','this', 'class', 'is', 'very', 'interesting']

x_ = np.zeros((1,10,50))

for ix in range(len(x)):
    x_[0][ix] = embeddings_index[x[ix].lower()]

In [26]:
model.predict_classes(x_)

array([3], dtype=int64)

##  - Using LSTM

In [27]:
# A simple LSTM network
model = Sequential()
model.add(LSTM(128, input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 128)           91648     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 223,877
Trainable params: 223,877
Non-trainable params: 0
________________________________________________

In [28]:
# Setting Loss ,Optimiser for model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
# Training model
hist = model.fit(embedding_matrix_train,Y_train,
                epochs = 50, batch_size=32,shuffle=True
                )

Epoch 1/50
5/5 [==============================] - 0s 16ms/step - loss: 1.5848 - accuracy: 0.2424
Epoch 2/50
5/5 [==============================] - 0s 13ms/step - loss: 1.5087 - accuracy: 0.2955
Epoch 3/50
5/5 [==============================] - 0s 11ms/step - loss: 1.4572 - accuracy: 0.3485
Epoch 4/50
5/5 [==============================] - 0s 11ms/step - loss: 1.3954 - accuracy: 0.4697
Epoch 5/50
5/5 [==============================] - 0s 11ms/step - loss: 1.2673 - accuracy: 0.5985
Epoch 6/50
5/5 [==============================] - 0s 11ms/step - loss: 1.1284 - accuracy: 0.6288
Epoch 7/50
5/5 [==============================] - 0s 11ms/step - loss: 1.0592 - accuracy: 0.5606
Epoch 8/50
5/5 [==============================] - 0s 11ms/step - loss: 0.9780 - accuracy: 0.5909
Epoch 9/50
5/5 [==============================] - 0s 11ms/step - loss: 0.8535 - accuracy: 0.7197
Epoch 10/50
5/5 [==============================] - 0s 11ms/step - loss: 0.7277 - accuracy: 0.7273
Epoch 11/50
5/5 [============

In [30]:
# Prediction of trained model
pred = model.predict_classes(embedding_matrix_test)

### ACCURACY

In [31]:
# Calculating accuracy / score  of the model
float(sum(pred==Y_test))/embedding_matrix_test.shape[0]

0.6607142857142857

In [32]:
# Printing the sentences with the predicted and the labelled emoji
for ix in range(embedding_matrix_test.shape[0]):
    
    if pred[ix] != Y_test[ix]:
        print(ix)
        print (test[0][ix],end=" ")
        print (emoji.emojize(emoji_dict[pred[ix]], use_aliases=True),end=" ")
        print (emoji.emojize(emoji_dict[Y_test[ix]], use_aliases=True))

3
['she', 'got', 'me', 'a', 'present'] 😄 ❤
5
['he', 'is', 'a', 'good', 'friend'] 😄 ❤
6
['I', 'am', 'upset'] 😞 ❤
7
['We', 'had', 'such', 'a', 'lovely', 'dinner', 'tonight'] 😄 ❤
11
['work', 'is', 'hard'] 😄 😞
12
['This', 'girl', 'is', 'messing', 'with', 'me'] ❤ 😞
16
['work', 'is', 'horrible'] 😄 😞
18
['stop', 'messing', 'around'] 🍴 😞
23
['she', 'is', 'a', 'bully'] ❤ 😞
26
['I', 'worked', 'during', 'my', 'birthday'] 😄 😞
28
['enjoy', 'your', 'break'] 😞 😄
29
['valentine', 'day', 'is', 'near'] 😄 ❤
32
['My', 'life', 'is', 'so', 'boring'] 😄 😞
34
['will', 'you', 'be', 'my', 'valentine'] 😄 ❤
40
['I', 'will', 'go', 'dance'] ⚾ 😄
41
['I', 'like', 'your', 'jacket'] ❤ 😄
45
['I', 'love', 'to', 'the', 'stars', 'and', 'back'] 😄 ❤
50
['yesterday', 'we', 'lost', 'again'] ⚾ 😞
55
['I', 'did', 'not', 'have', 'breakfast'] 🍴 😞
